# Quick Start 

Welcome to the **ASL(Agent Stucture Language)** — A DSL(Domain Specific Language) about how to building an agent — documentation! This page will give you an introduction to 80% of the ASL usage that you will use on a daily basis.

> You will learn:
>   1. How to build an agent by ASL.
>   2. How to reuse the exit agents in a componentized parttern.
>   3. How to build an agent with nested structure.
>   4. How to control data transmission in the agent
>   5. How to achieve dynamic topology during the agent's runtime.


## Introduction

ASL is a declarative language for agent construction. After all basic functions are modularly implemented, ASL pursue a what-you-see-is-what-you-get approach for the internal processes of the agent. We can clearly see the orchestration process and hierarchical structure at a glance.

### Build an Agent by ASL

Take the simplest text generation process as an example. When a user inputs a query, we break it down, and then generate text for every sub-query.

Let's first prepare necessary environment variables.

In [ ]:
# Get the environment variables.
import os

_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

# Import the necessary packages.
from typing import List, Dict
from bridgic.core.model.types import Message, Role
from bridgic.core.agentic.asl import ASLAutoma, graph
from bridgic.llms.openai import OpenAILlm, OpenAIConfiguration

llm = OpenAILlm(  # the llm instance
    api_base=_api_base,
    api_key=_api_key,
    timeout=5,
    configuration=OpenAIConfiguration(model=_model_name),
)

Secondly, modularly implement the functional functions needed in this process.

In [16]:
# Break down the query into a list of sub-queries.
async def break_down_query(user_input: str) -> List[str]:
    llm_response = await llm.achat(
        messages=[
            Message.from_text(text=f"Break down the query into multiple sub-queries and only return the sub-queries", role=Role.SYSTEM),
            Message.from_text(text=user_input, role=Role.USER,),
        ]
    )
    return [item.strip() for item in llm_response.message.content.split("\n") if item.strip()]

# Define the function to conduct a web search.
async def query_answer(queries: List[str]) -> Dict[str, str]:
    answers = []
    for query in queries:
        response = await llm.achat(
            messages=[
                Message.from_text(text=f"Answer the given query briefly", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER,),
            ]
        )
        answers.append(response.message.content)
    
    res = {
        query: answer
        for query, answer in zip(queries, answers)
    }
    return res

Now, Let's complete this process using ASL.

In [17]:
class SplitSolveAgent(ASLAutoma):
    with graph as g:
        a = break_down_query
        b = query_answer

        +a >> ~b

The implementation process of `SplitSolveAgent` was accomplished through orchestration using ASL grammar. In this grammar:
- `with graph as g`: Represents opening a graph, and we can **declare the nodes** and **defining the dependency** between the nodes under its syntax block.
- `a = break_down_query`: Represents declaring a node names `a`.
- `a >> b`: Represents defining the dependency of `b` is `a`, which means the `b` will execute after `a`.
- `+a`: Represents defining `a` is the start.
- `~b`: Represents defining `b` is the output.

Now, Let's run it!

In [ ]:
text_generation_agent = SplitSolveAgent()
await text_generation_agent.arun("When and where was the Einstein born?")

{'1. When was Einstein born?': 'Albert Einstein was born in 1879.',
 '2. Where was Einstein born?': 'Albert Einstein was born in Ulm, Kingdom of Württemberg, German Empire.'}

Great! We successfully obtained the result. In the ASL code, we can see very intuitively that the `SplitSolveAgent` has only two nodes, and `b` depends on `a`, with no other redundant information. ASL elevates node declarations and dependency management in an execution flow to first-class language constructs.

### Reuse the Exit Agents in Componentized Parttern

In the above process, we have completed the agent that splits the query and answers them separately. It is a pre-designed module. Now, I want to design a chatbot that merge these individual answers to generate a unified response to the original question. Like this:

In [20]:
async def merge_answers(qa_pairs: Dict[str, str], user_input: str) -> str:
    answers = "\n".join([v for v in qa_pairs.values()])
    llm_response = await llm.achat(
        messages=[
            Message.from_text(text=f"Merge the given answers into a unified response to the original question", role=Role.SYSTEM),
            Message.from_text(text=f"Query: {user_input}\nAnswers: {answers}", role=Role.USER,),
        ]
    )
    return llm_response.message.content

# Define the Chatbot agent, use SplitSolveAgent in componentized parttern.
class Chatbot(ASLAutoma):
    with graph as g:
        a = SplitSolveAgent()
        b = merge_answers

        +a >> ~b

Let's run it.

In [21]:
chatbot = Chatbot()
await chatbot.arun(user_input="When and where was the Einstein born?")

'Albert Einstein was born in 1879 in Ulm, Kingdom of Württemberg, German Empire.'

Great! Our chatbot successfully answered our questions. During its implementation, we directly declared `SplitSolveAgent` within the `with graph` statement block. We did not write a function to encapsulate it, nor did we use any form of API to manually add it to the graph. Everything was naturally declared and completed.

### Build an Agent with Nested Structure or Arranging Fregment

When we have all the functional function ready, but we don't `SplitSolveAgent`, we don't have to implement it specifically to reuse it. Instead, we can write everything in one agent directly.

In [ ]:
class ChatbotNested(ASLAutoma):
    with graph as g:  #  the chatbot agent to define its graph
        with graph as split_solve:  #  the split_solve agent to define its graph
            a = break_down_query
            b = query_answer    
            +a >> ~b
        end = merge_answers
        
        +split_solve >> ~end

Let's run it!

In [25]:
chatbot_nested = ChatbotNested()
await chatbot_nested.arun(user_input="When and where was the Einstein born?")

'Albert Einstein was born in 1879 in Ulm, Kingdom of Württemberg, German Empire.'

We successfully achieved the expected result. We can clearly see that there are two layers of graphs in `ChatbotNested` and what the arrangement structure of each graph is. 

Of course, the arrangement in one graph cannot use the nodes from another graph. In the above example, for graph `g`, nodes `a` and `b` are invisible, and it can only use `split_solve` as a whole to arrange beneath it. Similarly, for `split_solve`, it cannot see the nodes within other graphs either.

> Note: If a node that is unknown to the graph is referenced in it, an exception will be thrown.

### Control Data Transmission in the Agent

When executing an ASL code, the program first "translates" it into the corresponding objects in Bridgic. For instance, a node declared in ASL is actually a [`Worker`](../../../reference/bridgic-core/bridgic/core/automa/worker/index.md) during execution. Therefore, ASL also possesses all the underlying capabilities of Bridgic.

Bridgic offers a variety of rich [parameter binding mechanisms](../core_mechanism/parameter_binding.ipynb). In ASL, we can utilize them by setting the `Settings` and `Data` attributes of a node.

For example, the following code：

In [35]:
from bridgic.core.automa.args import ArgsMappingRule
from bridgic.core.agentic.asl import Settings


async def start1(user_input: int) -> int:
    return user_input + 1

async def start2(user_input: int) -> int:
    return user_input + 2

async def worker1(x: List[int], user_input: int) -> int:
    print(f"worker1: {x}, {user_input}")
    return sum(x) + user_input


class MyAgent(ASLAutoma):
    with graph as g:
        a = start1
        b = start2
        c = worker1 *Settings(args_mapping_rule=ArgsMappingRule.MERGE)

        +(a & b) >> ~c

The above code defines such a structure:

<br>
<div style="text-align: center;">
<img src="../../../imgs/asl_data_flow.png" alt="Parameter Passing" width="800" height="600">
</div>
<br>

In this grammar:
- `worker1 *Settings`: Represents that when adding the `worker1` function as an execution unit, some configurations are made for it. The field of `Settings` is the same as [the decorator `@worker` in bridgic](../quick_start/quick_start.ipynb#Worker). Now, `c` will receive result from `a` and `b` in the `ArgsMappingRule.MERGE` form.
- `+(a & b)`: Represents that the expressions for arranging control flow have distributive and associative laws. `(a & b)` indicates that `a` and `b` are arranged as a whole. `+(a & b)` indicates `a` and `b` are both start entry point.

Let's run it.

In [33]:
my_agent = MyAgent()
await my_agent.arun(user_input=1)

worker1: [2, 3], 1


6

In addition, bridgic also has a arguments injection mechanism, which can also be used in ASL through `Data`.

In [36]:
from bridgic.core.automa.args import From
from bridgic.core.agentic.asl import Data

class MyAgent(ASLAutoma):
    with graph as g:
        a = start1
        b = start2
        c = worker1 *Data(x=From('a'))

        +a >> b >> ~c

ValueError: non-default argument follows default argument